# Parking Lot Occupancy Estimation - Validation

This notebook evaluates trained models on the validation set with detailed metrics and visualizations.

**Author:** Aminu Yiwere  
**Date:** November 4, 2025  
**Environment:** Google Colab

---


## 1. Setup and Installation


In [ ]:
# Check if running on Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Running locally")

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install albumentations
!pip install timm
!pip install tqdm
!pip install matplotlib seaborn
!pip install scikit-learn
!pip install plotly

## 2. Import Libraries


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models

# Additional imports
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, classification_report,
    roc_curve, auc, roc_auc_score
)
import timm

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

## 3. Configuration


In [ ]:
# Configuration
class Config:
    # Paths
    if IN_COLAB:
        DATA_DIR = '/content/drive/MyDrive/parking_lot_data'
        OUTPUT_DIR = '/content/drive/MyDrive/parking_lot_output'
    else:
        DATA_DIR = './data/processed'
        OUTPUT_DIR = './output'
    
    VAL_DIR = os.path.join(DATA_DIR, 'validation')
    CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, 'checkpoints')
    RESULTS_DIR = os.path.join(OUTPUT_DIR, 'validation_results')
    
    # Model parameters
    MODEL_NAME = 'resnet50'  # Should match the trained model
    NUM_CLASSES = 2
    
    # Evaluation parameters
    BATCH_SIZE = 32
    IMG_SIZE = 224
    NUM_WORKERS = 2 if IN_COLAB else 4
    
    # Device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()
os.makedirs(config.RESULTS_DIR, exist_ok=True)

print(f"Device: {config.DEVICE}")
print(f"Model: {config.MODEL_NAME}")

## 4. Dataset and DataLoader


In [ ]:
# Custom Dataset Class
class ParkingLotDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = ['occupied', 'vacant']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        # Load image paths and labels
        self.samples = []
        for class_name in self.classes:
            class_dir = os.path.join(data_dir, class_name)
            if not os.path.exists(class_dir):
                continue
            
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, img_name)
                    label = self.class_to_idx[class_name]
                    self.samples.append((img_path, label))
        
        print(f"Found {len(self.samples)} images in {data_dir}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Apply transforms
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, label, img_path

# Validation transforms
def get_val_transforms(img_size=224):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ])

# Create dataset and dataloader
val_dataset = ParkingLotDataset(
    data_dir=config.VAL_DIR,
    transform=get_val_transforms(config.IMG_SIZE)
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    num_workers=config.NUM_WORKERS,
    pin_memory=True
)

print(f"Validation samples: {len(val_dataset)}")
print(f"Validation batches: {len(val_loader)}")

## 5. Load Trained Model


In [ ]:
def create_model(model_name, num_classes=2):
    """Create model architecture"""
    if model_name == 'resnet50':
        model = models.resnet50(pretrained=False)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, num_classes)
    
    elif model_name == 'resnet101':
        model = models.resnet101(pretrained=False)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, num_classes)
    
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=False)
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_features, num_classes)
    
    elif 'efficientnet' in model_name:
        model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    
    else:
        raise ValueError(f"Model {model_name} not supported")
    
    return model

# Load model
model = create_model(config.MODEL_NAME, config.NUM_CLASSES)
checkpoint_path = os.path.join(config.CHECKPOINT_DIR, f'best_model_{config.MODEL_NAME}.pth')

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=config.DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✓ Model loaded from {checkpoint_path}")
    print(f"  Trained for {checkpoint['epoch'] + 1} epochs")
    print(f"  Validation Loss: {checkpoint['val_loss']:.4f}")
    print(f"  Validation Accuracy: {checkpoint['val_acc']:.4f}")
else:
    print(f"⚠ Checkpoint not found at {checkpoint_path}")
    print("Please train the model first using train.ipynb")

model = model.to(config.DEVICE)
model.eval()
print("\nModel ready for evaluation!")

## 6. Evaluation Function


In [ ]:
def evaluate_model(model, dataloader, device):
    """
    Comprehensive model evaluation
    """
    model.eval()
    
    all_preds = []
    all_labels = []
    all_probs = []
    all_paths = []
    
    with torch.no_grad():
        for images, labels, paths in tqdm(dataloader, desc='Evaluating'):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            # Store predictions
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_paths.extend(paths)
    
    return np.array(all_preds), np.array(all_labels), np.array(all_probs), all_paths

## 7. Run Evaluation


In [ ]:
# Evaluate model
predictions, labels, probabilities, image_paths = evaluate_model(model, val_loader, config.DEVICE)

print("\nEvaluation complete!")
print(f"Total samples evaluated: {len(predictions)}")

## 8. Calculate Metrics


In [ ]:
# Calculate metrics
accuracy = accuracy_score(labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
conf_matrix = confusion_matrix(labels, predictions)

# Calculate per-class metrics
precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
    labels, predictions, average=None
)

# ROC AUC
roc_auc = roc_auc_score(labels, probabilities[:, 1])

print("\n" + "="*50)
print("EVALUATION METRICS")
print("="*50)
print(f"\nOverall Metrics:")
print(f"  Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"  ROC AUC:   {roc_auc:.4f}")

print(f"\nPer-Class Metrics:")
print(f"  Class 0 (Occupied):")
print(f"    Precision: {precision_per_class[0]:.4f}")
print(f"    Recall:    {recall_per_class[0]:.4f}")
print(f"    F1-Score:  {f1_per_class[0]:.4f}")
print(f"    Support:   {support[0]}")

print(f"\n  Class 1 (Vacant):")
print(f"    Precision: {precision_per_class[1]:.4f}")
print(f"    Recall:    {recall_per_class[1]:.4f}")
print(f"    F1-Score:  {f1_per_class[1]:.4f}")
print(f"    Support:   {support[1]}")
print("="*50)

## 9. Confusion Matrix Visualization


In [ ]:
def plot_confusion_matrix(conf_matrix, class_names=['Occupied', 'Vacant']):
    """
    Plot confusion matrix with annotations
    """
    plt.figure(figsize=(10, 8))
    
    # Normalize confusion matrix
    conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
    
    # Plot
    sns.heatmap(
        conf_matrix_normalized,
        annot=True,
        fmt='.2%',
        cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names,
        square=True,
        cbar_kws={'label': 'Percentage'}
    )
    
    # Add counts
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.7, f'({conf_matrix[i, j]})',
                    ha='center', va='center', color='gray', fontsize=10)
    
    plt.title('Confusion Matrix (Normalized)', fontsize=16, pad=20)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(os.path.join(config.RESULTS_DIR, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
    plt.show()

plot_confusion_matrix(conf_matrix)

## 10. ROC Curve


In [ ]:
def plot_roc_curve(labels, probabilities):
    """
    Plot ROC curve
    """
    fpr, tpr, thresholds = roc_curve(labels, probabilities[:, 1])
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16, pad=20)
    plt.legend(loc='lower right', fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(config.RESULTS_DIR, 'roc_curve.png'), dpi=300, bbox_inches='tight')
    plt.show()

plot_roc_curve(labels, probabilities)

## 11. Classification Report


In [ ]:
# Print classification report
print("\nDetailed Classification Report:")
print("="*60)
report = classification_report(
    labels, 
    predictions, 
    target_names=['Occupied', 'Vacant'],
    digits=4
)
print(report)

# Save to file
with open(os.path.join(config.RESULTS_DIR, 'classification_report.txt'), 'w') as f:
    f.write(report)

## 12. Error Analysis - Misclassified Samples


In [ ]:
# Find misclassified samples
misclassified_indices = np.where(predictions != labels)[0]
print(f"\nTotal misclassified samples: {len(misclassified_indices)}")
print(f"Misclassification rate: {len(misclassified_indices)/len(labels)*100:.2f}%")

# Visualize some misclassified samples
def visualize_misclassified(indices, num_samples=12):
    if len(indices) == 0:
        print("No misclassified samples found!")
        return
    
    num_samples = min(num_samples, len(indices))
    sample_indices = np.random.choice(indices, num_samples, replace=False)
    
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.ravel()
    
    class_names = ['Occupied', 'Vacant']
    
    for i, idx in enumerate(sample_indices):
        img_path = image_paths[idx]
        true_label = labels[idx]
        pred_label = predictions[idx]
        confidence = probabilities[idx][pred_label]
        
        # Load and display image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        axes[i].imshow(img)
        axes[i].set_title(
            f"True: {class_names[true_label]}\n"
            f"Pred: {class_names[pred_label]} ({confidence:.2%})",
            fontsize=10,
            color='red'
        )
        axes[i].axis('off')
    
    plt.suptitle('Misclassified Samples', fontsize=16, y=1.00)
    plt.tight_layout()
    plt.savefig(os.path.join(config.RESULTS_DIR, 'misclassified_samples.png'), dpi=300, bbox_inches='tight')
    plt.show()

if len(misclassified_indices) > 0:
    visualize_misclassified(misclassified_indices)

## 13. Confidence Distribution Analysis


In [ ]:
# Analyze confidence distribution
correct_confidences = [probabilities[i][predictions[i]] for i in range(len(predictions)) if predictions[i] == labels[i]]
incorrect_confidences = [probabilities[i][predictions[i]] for i in range(len(predictions)) if predictions[i] != labels[i]]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Correct predictions
axes[0].hist(correct_confidences, bins=30, color='green', alpha=0.7, edgecolor='black')
axes[0].set_title(f'Confidence Distribution - Correct Predictions\n(n={len(correct_confidences)})', fontsize=12)
axes[0].set_xlabel('Confidence', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].axvline(np.mean(correct_confidences), color='darkgreen', linestyle='--', linewidth=2, label=f'Mean: {np.mean(correct_confidences):.3f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Incorrect predictions
if len(incorrect_confidences) > 0:
    axes[1].hist(incorrect_confidences, bins=30, color='red', alpha=0.7, edgecolor='black')
    axes[1].set_title(f'Confidence Distribution - Incorrect Predictions\n(n={len(incorrect_confidences)})', fontsize=12)
    axes[1].set_xlabel('Confidence', fontsize=11)
    axes[1].set_ylabel('Frequency', fontsize=11)
    axes[1].axvline(np.mean(incorrect_confidences), color='darkred', linestyle='--', linewidth=2, label=f'Mean: {np.mean(incorrect_confidences):.3f}')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
else:
    axes[1].text(0.5, 0.5, 'No incorrect predictions!', ha='center', va='center', fontsize=14)
    axes[1].set_xlim([0, 1])

plt.tight_layout()
plt.savefig(os.path.join(config.RESULTS_DIR, 'confidence_distribution.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"\nConfidence Statistics:")
print(f"  Correct predictions - Mean: {np.mean(correct_confidences):.4f}, Std: {np.std(correct_confidences):.4f}")
if len(incorrect_confidences) > 0:
    print(f"  Incorrect predictions - Mean: {np.mean(incorrect_confidences):.4f}, Std: {np.std(incorrect_confidences):.4f}")

## 14. Save Results


In [ ]:
# Create results DataFrame
results_df = pd.DataFrame({
    'image_path': image_paths,
    'true_label': labels,
    'predicted_label': predictions,
    'correct': predictions == labels,
    'confidence': [probabilities[i][predictions[i]] for i in range(len(predictions))],
    'prob_occupied': probabilities[:, 0],
    'prob_vacant': probabilities[:, 1]
})

# Save to CSV
results_csv_path = os.path.join(config.RESULTS_DIR, 'validation_results.csv')
results_df.to_csv(results_csv_path, index=False)
print(f"\n✓ Results saved to {results_csv_path}")

# Save metrics summary
metrics_dict = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC AUC'],
    'Score': [accuracy, precision, recall, f1, roc_auc]
}
metrics_df = pd.DataFrame(metrics_dict)
metrics_csv_path = os.path.join(config.RESULTS_DIR, 'metrics_summary.csv')
metrics_df.to_csv(metrics_csv_path, index=False)
print(f"✓ Metrics summary saved to {metrics_csv_path}")

print("\n" + "="*50)
print("VALIDATION COMPLETE!")
print("="*50)
print(f"All results saved to: {config.RESULTS_DIR}")

## 15. Summary


In [ ]:
# Print final summary
print("\n" + "="*60)
print("VALIDATION SUMMARY")
print("="*60)
print(f"\nModel: {config.MODEL_NAME}")
print(f"Total Samples: {len(labels)}")
print(f"Correct Predictions: {np.sum(predictions == labels)} ({accuracy*100:.2f}%)")
print(f"Incorrect Predictions: {np.sum(predictions != labels)} ({(1-accuracy)*100:.2f}%)")
print(f"\nKey Metrics:")
print(f"  • Accuracy:  {accuracy:.4f}")
print(f"  • Precision: {precision:.4f}")
print(f"  • Recall:    {recall:.4f}")
print(f"  • F1-Score:  {f1:.4f}")
print(f"  • ROC AUC:   {roc_auc:.4f}")
print("\n" + "="*60)